In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

In [2]:
Data = pd.read_csv('Dataset-Unilever.csv')

In [4]:
Data.columns

Index([' YEAR ', ' MONTH ', ' SALESMAN_CODE ', ' channel ', ' subchannel ',
       ' CUSTOMER_CODE ', ' ORDERNO ', ' DISTRICTNAMEE ', ' CITYNAMEE ',
       ' AREANAMEE ', ' TOT_CATEGORY_DESC ', ' CATEGORY_DESC ', ' BRAND_DESC ',
       ' Chiffre d'affaire ', ' KD ', ' Active ', ' Customer Address ',
       ' Lattitude ', ' Longitude ', ' Saturday ', ' Sunday ', ' Monday ',
       ' Tuesday ', ' Wednesday ', ' Thursday ', ' Friday ', ' Selling Type '],
      dtype='object')

We group them first by the customer_code, we suppose that we have a uniquer costumer code per area and this is confirmed through the follwing

In [5]:
for i in range(Data[' CUSTOMER_CODE '].unique().shape[0]):
    if (Data[Data[' CUSTOMER_CODE '] == Data[' CUSTOMER_CODE '].unique()[1570]][" AREANAMEE "].unique().shape[0] !=1):
        print('The theory is wrong')
    

In [6]:
df = Data.sort_values(by = ' CUSTOMER_CODE ')
df.drop(' DISTRICTNAMEE ',axis=1 ,inplace=True)
df.drop(' YEAR ',axis=1, inplace=True) 

In [8]:
def calculate_products(visit_plan_code):
    if visit_plan_code == 0:
        return [0, 0, 0, 0]  # No visit
    elif visit_plan_code == 1:
        return [1, 0, 1, 0] 
    elif visit_plan_code == 2:
        return [0, 1, 0, 1] # Example value for monthly visits
    elif visit_plan_code == 3:
        return [1, 1, 1, 1]
    elif visit_plan_code == 4:
        return [1, 0, 0, 0]
    elif visit_plan_code == 5:
        return [0, 1, 0, 0]
    elif visit_plan_code == 6:
        return [0, 0, 1, 0]
    elif visit_plan_code == 7:
        return [0, 0, 0, 1]
    else:
        pass

# Example customer code
customer_code = 350110245

# Filter data for the specific customer
customer_data = df[df[' CUSTOMER_CODE '] == customer_code]



In [9]:
customer_data

,MONTH,SALESMAN_CODE,channel,subchannel,CUSTOMER_CODE,ORDERNO,CITYNAMEE,AREANAMEE,TOT_CATEGORY_DESC,CATEGORY_DESC,BRAND_DESC,Chiffre d'affaire,KD,Active,Customer Address,Lattitude,Longitude,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Selling Type
85456,3.0,3501084.0,20.0,201.0,350110245.0,"3,501,368,317.00",Baraki,Baraki,Home Care,OMO,OMO HS,"2,308.35",ALGER/BOUMERDES,1.00,Cité amirat baraki,36.64,3.11,0.0,0.0,0.0,0.0,0.0,3.0,0.0,HPC
66421,2.0,3501084.0,20.0,201.0,350110245.0,"3,501,364,276.00",Baraki,Baraki,Home Care,OMO,OMO HS,"2,296.33",ALGER/BOUMERDES,1.00,Cité amirat baraki,36.64,3.11,0.0,0.0,0.0,0.0,0.0,3.0,0.0,HPC
85457,3.0,3501084.0,20.0,201.0,350110245.0,"3,501,368,317.00",Baraki,Baraki,Home Care,SURF,SURF HS,"1,808.47",ALGER/BOUMERDES,1.00,Cité amirat baraki,36.64,3.11,0.0,0.0,0.0,0.0,0.0,3.0,0.0,HPC
85455,3.0,3501084.0,20.0,201.0,350110245.0,"3,501,368,317.00",Baraki,Baraki,Home Care,OMO,OMO HS,"1,951.33",ALGER/BOUMERDES,1.00,Cité amirat baraki,36.64,3.11,0.0,0.0,0.0,0.0,0.0,3.0,0.0,HPC
66420,2.0,3501084.0,20.0,201.0,350110245.0,"3,501,364,276.00",Baraki,Baraki,Home Care,OMO,OMO HS,"2,331.33",ALGER/BOUMERDES,1.00,Cité amirat baraki,36.64,3.11,0.0,0.0,0.0,0.0,0.0,3.0,0.0,HPC


In [10]:
products_per_area = {}
# we need to add the month specification
# Iterate over days of the week
for day in [' Sunday ', ' Monday ', ' Tuesday ', ' Wednesday ', ' Thursday ', ' Friday ', ' Saturday ']:
    # Get visit plan code for the day
    visit_plan_code = customer_data[day].iloc[4]  # Assuming same plan for all rows of the same customer
    
    # Calculate products for the day based on visit plan code
    products = calculate_products(visit_plan_code)
    print(products)
    

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [11]:
weekdays = [' Sunday ', ' Monday ', ' Tuesday ', ' Wednesday ', ' Thursday ', ' Friday ', ' Saturday ']
dft = df.copy()

In [12]:
for day in weekdays:
    df[day] = df[day].apply(calculate_products)
dft = df.copy()

In [14]:
data_final = pd.DataFrame()
for m in dft[' MONTH '].unique():
    # Preparing the dataset
    df = dft.copy()
    # dft.set_index(' CUSTOMER_CODE ',inplace=True)
    df = df[df[' MONTH ']==m]
    #df.drop(' MONTH ', axis=1, inplace=True)
    df.head()
    for customer_code in df[' CUSTOMER_CODE '].unique():
        df_new = df[df[' CUSTOMER_CODE ']==customer_code]
        for day in weekdays:
            product_per_single_day_and_customer = df_new[day].apply(np.array).sum()
            for id in df[df[' CUSTOMER_CODE ']==customer_code][day].index:
                df[df[' CUSTOMER_CODE ']==customer_code][day][id] = product_per_single_day_and_customer
    data_final = pd.concat([data_final, df])



/tmp/ipykernel_49258/2849127938.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df[' CUSTOMER_CODE ']==customer_code][day][id] = product_per_single_day_and_customer


In [15]:
dft[' CUSTOMER_CODE '].unique()

array([9.02030250e+07, 9.02030450e+07, 9.02030490e+07, ...,
       3.50110243e+08, 3.50110244e+08, 3.50110245e+08])

In [16]:
df = data_final

In [17]:
df.sort_values(by=' AREANAMEE ')[' Longitude '].unique()

array([' 2.94 ', ' 2.90 ', ' 2.92 ', ' 2.91 ', ' 2.93 ', ' 36.79 ',
       ' 2.98 ', ' 3.29 ', ' 3.28 ', ' 3.31 ', ' 3.30 ', ' 3.25 ',
       ' 3.26 ', ' 3.06 ', ' 3.05 ', ' -   ', ' 3.02 ', ' 3.09 ',
       ' 3.18 ', ' 3.17 ', ' 3.20 ', ' 3.19 ', ' 3.22 ', ' 36.78 ',
       ' 3.24 ', ' 2.96 ', ' 2.97 ', ' 2.99 ', ' 2.95 ', ' 3.12 ',
       ' 3.11 ', ' 3.08 ', ' 3.10 ', ' 3.07 ', ' 7.36 ', ' 3.13 ',
       ' 3.01 ', ' 3.00 ', ' 3.04 ', ' 3.03 ', ' 3.23 ', ' 3.21 ',
       ' 3.27 ', ' 36.76 ', ' 3.15 ', ' 3.14 ', ' 3.16 ', ' 2.81 ',
       ' 3.32 ', ' 3.33 ', ' 2.85 ', ' 2.88 ', ' 2.84 ', ' 2.86 ',
       ' 2.87 ', ' 2.89 ', ' 3.35 ', ' 3.36 ', ' 3.34 ', ' 3.37 ',
       ' 36.74 ', ' 2.83 '], dtype=object)

In [123]:
df = df.sort_values(by=' AREANAMEE ')
df.loc[df[' Lattitude '] == ' -   ',[' Lattitude ']] = ' 36.79 '
df.loc[df[' Longitude '] == ' -   ',[' Longitude ']] = ' 2.94 '


In [124]:
df[' Lattitude '].unique()

array([' 36.79 ', ' 36.80 ', ' 36.68 ', ' 2.91 ', ' 36.78 ', ' 36.72 ',
       ' 36.77 ', ' 36.76 ', ' 36.74 ', ' 3.25 ', ' 36.73 ', ' 36.71 ',
       ' 36.69 ', ' 36.70 ', ' 36.67 ', ' 36.65 ', ' 36.66 ', ' 41.13 ',
       ' 36.64 ', ' 36.75 ', ' 36.59 ', ' 36.61 ', ' 36.62 ', ' 36.82 ',
       ' 36.81 ', ' 36.58 ', ' 36.63 ', ' 36.55 ', ' 36.60 ', ' 3.06 '],
      dtype=object)

In [126]:
filtered = df.loc[:, [' MONTH ',' CUSTOMER_CODE ', ' AREANAMEE ',' Lattitude ', ' Longitude ', ' Saturday ', ' Sunday ', ' Monday ', ' Tuesday ', ' Wednesday ', ' Thursday ', ' Friday ']].astype('str').drop_duplicates()
filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6544 entries, 50033 to 27980
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0    MONTH           6544 non-null   object
 1    CUSTOMER_CODE   6544 non-null   object
 2    AREANAMEE       6544 non-null   object
 3    Lattitude       6544 non-null   object
 4    Longitude       6544 non-null   object
 5    Saturday        6544 non-null   object
 6    Sunday          6544 non-null   object
 7    Monday          6544 non-null   object
 8    Tuesday         6544 non-null   object
 9    Wednesday       6544 non-null   object
 10   Thursday        6544 non-null   object
 11   Friday          6544 non-null   object
dtypes: object(12)
memory usage: 664.6+ KB


In [127]:
filtered = filtered[filtered[' MONTH '] == str(1.0)]

In [128]:
filtered[' MONTH '].value_counts()

 MONTH 
1.0    2372
Name: count, dtype: int64

In [129]:
filtered.head()

,MONTH,CUSTOMER_CODE,AREANAMEE,Lattitude,Longitude,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday
5855,1.0,350106856.0,A?n Benian,36.79,2.92,"[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
3527,1.0,350104130.0,A?n Benian,36.80,2.91,"[0, 0, 0, 0]","[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
23475,1.0,350104132.0,A?n Benian,36.79,2.92,"[0, 0, 0, 0]","[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
30190,1.0,350107375.0,A?n Benian,36.79,2.91,"[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
14929,1.0,350106841.0,A?n Benian,36.80,2.93,"[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"


In [130]:
import json
columns_to = [' Saturday ', ' Sunday ', ' Monday ', ' Tuesday ', ' Wednesday ', ' Thursday ', ' Friday ']
for clm in columns_to:
    for i in range(len(filtered)):
        array = json.loads(filtered.loc[:, [clm]].values[i][0])
        filtered[str(clm) + ' - First Week'] = array[0]
        filtered[str(clm) + ' - Second Week'] = array[1]
        filtered[str(clm) + ' - Third Week'] = array[2]
        filtered[str(clm) + ' - Forth Week'] = array[3]

In [131]:
filtered = filtered.drop(columns_to, axis=1)

In [132]:
filtered.tail()

,MONTH,CUSTOMER_CODE,AREANAMEE,Lattitude,Longitude,Saturday - First Week,Saturday - Second Week,Saturday - Third Week,Saturday - Forth Week,Sunday - First Week,Sunday - Second Week,Sunday - Third Week,Sunday - Forth Week,Monday - First Week,Monday - Second Week,Monday - Third Week,Monday - Forth Week,Tuesday - First Week,Tuesday - Second Week,Tuesday - Third Week,Tuesday - Forth Week,Wednesday - First Week,Wednesday - Second Week,Wednesday - Third Week,Wednesday - Forth Week,Thursday - First Week,Thursday - Second Week,Thursday - Third Week,Thursday - Forth Week,Friday - First Week,Friday - Second Week,Friday - Third Week,Friday - Forth Week
12371,1.0,350107805.0,Zeralda,36.71,2.84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
27979,1.0,350107811.0,Zeralda,36.71,2.83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
22978,1.0,90203118.0,Zeralda,36.71,2.84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
32893,1.0,90203117.0,Zeralda,36.71,2.84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
27980,1.0,350107815.0,Zeralda,36.71,2.84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0


In [38]:
test = filtered.drop_duplicates(subset=[' AREANAMEE '])

In [39]:
test.shape

(59, 31)

In [133]:
sorted_df = filtered.sort_values(by=' AREANAMEE ')
len(sorted_df[' AREANAMEE '])

2372

In [134]:
np.sum(sorted_df.loc[sorted_df[' AREANAMEE '] == ' A?n Benian ', [' Saturday  - Third Week']].values)

0

In [135]:
sorted_df.columns

Index([' MONTH ', ' CUSTOMER_CODE ', ' AREANAMEE ', ' Lattitude ',
       ' Longitude ', ' Saturday  - First Week', ' Saturday  - Second Week',
       ' Saturday  - Third Week', ' Saturday  - Forth Week',
       ' Sunday  - First Week', ' Sunday  - Second Week',
       ' Sunday  - Third Week', ' Sunday  - Forth Week',
       ' Monday  - First Week', ' Monday  - Second Week',
       ' Monday  - Third Week', ' Monday  - Forth Week',
       ' Tuesday  - First Week', ' Tuesday  - Second Week',
       ' Tuesday  - Third Week', ' Tuesday  - Forth Week',
       ' Wednesday  - First Week', ' Wednesday  - Second Week',
       ' Wednesday  - Third Week', ' Wednesday  - Forth Week',
       ' Thursday  - First Week', ' Thursday  - Second Week',
       ' Thursday  - Third Week', ' Thursday  - Forth Week',
       ' Friday  - First Week', ' Friday  - Second Week',
       ' Friday  - Third Week', ' Friday  - Forth Week'],
      dtype='object')

In [136]:
x = pd.DataFrame(sorted_df.drop_duplicates(subset=[' AREANAMEE '])[' AREANAMEE '].values, columns=['Area Name']).reset_index(0)
x.shape

(59, 2)

In [137]:
y = sorted_df.drop_duplicates(subset=[' AREANAMEE ']).loc[:, sorted_df.columns[3:]].reset_index(0)
y.shape

(59, 31)

In [138]:
sorted_clean = pd.concat([x, y], axis=1).reset_index(0)
columns_to_put = [' Saturday  - First Week', ' Saturday  - Second Week',
       ' Saturday  - Third Week', ' Saturday  - Forth Week',
       ' Sunday  - First Week', ' Sunday  - Second Week',
       ' Sunday  - Third Week', ' Sunday  - Forth Week',
       ' Monday  - First Week', ' Monday  - Second Week',
       ' Monday  - Third Week', ' Monday  - Forth Week',
       ' Tuesday  - First Week', ' Tuesday  - Second Week',
       ' Tuesday  - Third Week', ' Tuesday  - Forth Week',
       ' Wednesday  - First Week', ' Wednesday  - Second Week',
       ' Wednesday  - Third Week', ' Wednesday  - Forth Week',
       ' Thursday  - First Week', ' Thursday  - Second Week',
       ' Thursday  - Third Week', ' Thursday  - Forth Week',
       ' Friday  - First Week', ' Friday  - Second Week',
       ' Friday  - Third Week', ' Friday  - Forth Week']

In [139]:
sorted_clean.shape

(59, 34)

In [140]:
for area in sorted_clean['Area Name'].unique():
    for column in columns_to_put:
        sorted_clean.loc[sorted_clean['Area Name'] == area,[column]] = np.sum(sorted_df.loc[sorted_df[' AREANAMEE '] == area, :].loc[:, [column]].values)

In [141]:
len(sorted_clean['Area Name'])

59

In [145]:
sorted_clean.columns

Index(['Area Name', ' Lattitude ', ' Longitude ', ' Saturday  - First Week',
       ' Saturday  - Second Week', ' Saturday  - Third Week',
       ' Saturday  - Forth Week', ' Sunday  - First Week',
       ' Sunday  - Second Week', ' Sunday  - Third Week',
       ' Sunday  - Forth Week', ' Monday  - First Week',
       ' Monday  - Second Week', ' Monday  - Third Week',
       ' Monday  - Forth Week', ' Tuesday  - First Week',
       ' Tuesday  - Second Week', ' Tuesday  - Third Week',
       ' Tuesday  - Forth Week', ' Wednesday  - First Week',
       ' Wednesday  - Second Week', ' Wednesday  - Third Week',
       ' Wednesday  - Forth Week', ' Thursday  - First Week',
       ' Thursday  - Second Week', ' Thursday  - Third Week',
       ' Thursday  - Forth Week', ' Friday  - First Week',
       ' Friday  - Second Week', ' Friday  - Third Week',
       ' Friday  - Forth Week'],
      dtype='object')

In [144]:
sorted_clean.drop(['level_0', 'index','index'], axis=1, inplace=True)

In [146]:
sorted_clean.to_csv('Unilever_cleaned.csv', index=False)